<a href="https://colab.research.google.com/github/Basit2612/Bidirectional-LSTM-RNN/blob/main/Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label', axis=1)

In [ ]:
y = df['label']

In [ ]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
voc_size=5000

In [ ]:
message = X.copy()

In [ ]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
message.reset_index(inplace=True)

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(message)):
  review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
sent_length = 20
one_hot_regr = [one_hot(words, voc_size) for words in corpus]
one_hot_regr

[[4173, 2802, 3781, 1279, 22, 788, 383, 154, 984, 2497],
 [1032, 2420, 4805, 1699, 1848, 3854, 3237],
 [40, 716, 1645, 51],
 [3868, 3706, 504, 772, 4537, 4434],
 [3751, 1848, 1326, 503, 553, 1966, 1848, 3759, 3456, 1471],
 [1991,
  1963,
  3645,
  1619,
  4522,
  2828,
  2398,
  3130,
  4545,
  4749,
  4267,
  3989,
  2215,
  1985,
  3237],
 [4201, 2759, 2144, 2587, 2301, 506, 1661, 772, 2960, 1927, 4293],
 [995, 2156, 4298, 4364, 4464, 4243, 2828, 4629, 2960, 1927, 4293],
 [2964, 2923, 4533, 1334, 3747, 2935, 1818, 4434, 2828, 4857],
 [564, 3772, 4596, 3419, 3858, 4131, 2755, 4209],
 [2032, 1835, 3932, 4077, 3732, 4043, 834, 3090, 1852, 2689, 1422],
 [772, 4367, 22, 2935, 2828, 4464],
 [2071, 4794, 3186, 4817, 2078, 361, 285, 3195, 2349],
 [1154, 621, 1068, 3936, 3849, 3836, 2571, 2960, 1927, 4293],
 [3307, 780, 3750, 2916, 960, 2960, 1927, 4293],
 [4604, 1793, 3703, 4842, 1642, 3144, 4359, 3719, 2377, 3698],
 [4176, 4622, 2420],
 [4099, 3030, 1094, 4883, 2828, 4668, 3557, 3237],
 [17

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_regr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  154  984 2497]
 [   0    0    0 ... 1848 3854 3237]
 [   0    0    0 ...  716 1645   51]
 ...
 [   0    0    0 ... 2960 1927 4293]
 [   0    0    0 ... 2026 4291 4506]
 [   0    0    0 ... 4112 1918 3635]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4173,
       2802, 3781, 1279,   22,  788,  383,  154,  984, 2497], dtype=int32)

In [ ]:
embedding_vector_feature=40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=32)

Epoch 1/10
383/383 [==============================] - 31s 63ms/step - loss: 0.2582 - accuracy: 0.8834 - val_loss: 0.1975 - val_accuracy: 0.9148
Epoch 2/10
383/383 [==============================] - 23s 60ms/step - loss: 0.1322 - accuracy: 0.9505 - val_loss: 0.2053 - val_accuracy: 0.9220
Epoch 3/10
383/383 [==============================] - 23s 61ms/step - loss: 0.0854 - accuracy: 0.9696 - val_loss: 0.2021 - val_accuracy: 0.9201
Epoch 4/10
383/383 [==============================] - 24s 63ms/step - loss: 0.0526 - accuracy: 0.9820 - val_loss: 0.2872 - val_accuracy: 0.9188
Epoch 5/10
383/383 [==============================] - 17s 46ms/step - loss: 0.0345 - accuracy: 0.9885 - val_loss: 0.3644 - val_accuracy: 0.9123
Epoch 6/10
383/383 [==============================] - 15s 38ms/step - loss: 0.0249 - accuracy: 0.9928 - val_loss: 0.4229 - val_accuracy: 0.9070
Epoch 7/10
383/383 [==============================] - 14s 38ms/step - loss: 0.0157 - accuracy: 0.9953 - val_loss: 0.3977 - val_accuracy:

In [ ]:
y_pred = model.predict(X_test)

189/189 [==============================] - 2s 8ms/step


In [ ]:
import numpy as np
y_pred = np.where(y_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_pred)

array([[3124,  295],
       [ 210, 2406]])

In [ ]:
from sklearn.metrics import classification_report
classification_report(Y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.94      0.91      0.93      3419\n           1       0.89      0.92      0.91      2616\n\n    accuracy                           0.92      6035\n   macro avg       0.91      0.92      0.92      6035\nweighted avg       0.92      0.92      0.92      6035\n'

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.9163214581607291